<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AGENT_ORCHESTRARION_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Mar 22 20:03:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   47C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install --upgrade crewai langchain -q
!pip install --upgrade colab-env --quiet
!pip install --upgrade langchain_community -q
!pip install --upgrade faiss-cpu -q
!pip install --upgrade crewai-tools -q
!pip install transformers -U -q

In [ ]:
import warnings

warnings.filterwarnings("ignore", message="You seem to be using the pipelines sequentially on GPU")

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [2]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_community.llms import HuggingFaceHub

In [3]:
import colab_env
import os

In [4]:
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
from crewai import Agent, Task, Crew, Process
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
import torch
from langchain.llms import HuggingFaceHub
import os
import colab_env

# Retrieve the token from the environment variable, if available
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

# If access_token_write is not set, fallback to using the HUGGINGFACEHUB_API_TOKEN env variable
if access_token_write is None:
    access_token_write = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
else:
   os.environ["MODEL_PROVIDER_HUGGINGFACE"] = "true"
   # Set the MODEL_PROVIDER environment variable to tell litellm to use Hugging Face Hub
   os.environ["MODEL_PROVIDER"] = "huggingface"

   print("THE HUGGINGFACE_ACCESS_TOKEN_WRITE IS WORKING ..... ")
   print('\n')

# If neither is set, prompt the user to enter the token manually
if access_token_write is None:
    access_token_write = input("Enter your Hugging Face API token: ")


# Initialize the HuggingFaceHub LLM using the token
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"  # Or your preferred model
hf_llm = HuggingFaceHub(
    repo_id=model_name,
    model_kwargs={"temperature": 0.1, "max_length": 200},
    huggingfacehub_api_token=access_token_write  # Pass the token here
)


# 2. Define Agents
researcher = Agent(
    role='Senior Research Analyst',
    goal='Conduct thorough research on a given topic using only the provided LLM.',
    backstory="You are a seasoned research analyst with a knack for finding key information.",
    allow_delegation=False,
    verbose=True,
    llm=hf_llm,  # Assign the Hugging Face Hub LLM
)


writer = Agent(
    role='Technical Writer',
    goal='Write a clear and concise tutorial based on research using only the provided LLM.',
    backstory="You are an expert technical writer, skilled in simplifying complex topics.",
    allow_delegation=False,
    verbose=True,
    llm=hf_llm,  # Assign the Hugging Face Hub LLM
)

# 3. Define Tasks
research_task = Task(
    description="Research the basics of crewAI, focusing on agents, tasks, and crews.",
    agent=researcher,
    expected_output="A concise summary of crewAI, agents, tasks, and crews.",
)

write_tutorial_task = Task(
    description="Write a short tutorial explaining crewAI using the research provided.",
    agent=writer,
    expected_output="A short tutorial explaining crewAI concepts.",
)

# 4. Form the Crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_tutorial_task],
    process=Process.sequential,
    verbose=True,
)

# 5. Kickoff the Crew
result = crew.kickoff()

# 6. Print the Result
print("\n\nTutorial using Hugging Face Hub LLM:\n")
print(result)